In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from prophet import Prophet


# Load Dataset
data = pd.read_csv("C:\GitHub\Machine-Learning\data\Payment Data.csv")

# dataset preprocess
data['ACC_MONTH'] = pd.to_datetime(data['ACC_MONTH'])
df = data[['ACC_MONTH', 'TOT_PAID', 'EST_COST', 'NBR_CASE', 'NBR_KEY_CASE','NBR_OTHER']].dropna()
df = data.sort_values(by='ACC_MONTH')

df['EST_COST_lag12'] = df['EST_COST'].shift(12)


In [25]:
df.head()

,ACC_MONTH,TOT_PAID,EST_COST,NBR_CASE,NBR_KEY_CASE,NBR_OTHER,EST_COST_lag12
0,2011-04-30,1489878.86,3629805.06,62,16,46,NaN
1,2011-05-31,1922713.79,1724956.61,41,8,33,NaN
2,2011-06-30,2378873.09,2623665.50,38,12,26,NaN
3,2011-07-31,1853901.62,2626410.66,49,9,40,NaN
4,2011-08-31,2024100.54,1995629.23,33,17,16,NaN


In [30]:
def create_shifted_regressor_df(df, regressor, shift_months=12):
    df_shifted = df[['ACC_MONTH', regressor]].copy()
    df_shifted['ACC_MONTH'] = df_shifted['ACC_MONTH'] + pd.DateOffset(months=shift_months)
    df_shifted = df_shifted.rename(columns={regressor: f'{regressor}_lag{shift_months}'})
    return df_shifted
# List of regressors
regressors = ['EST_COST', 'NBR_OTHER']
shift_months = 12

# Create shifted regressor DataFrames
shifted_reg_dfs = [create_shifted_regressor_df(data, reg, shift_months) for reg in regressors]

# Combine them into one DataFrame by merging on shifted ACC_MONTH
from functools import reduce
regressors_lagged_df = reduce(lambda left, right: pd.merge(left, right, on='ACC_MONTH', how='outer'), shifted_reg_dfs)

# Sort and reset index
regressors_lagged_df = regressors_lagged_df.sort_values('ACC_MONTH').reset_index(drop=True)

In [31]:


regressors_lagged_df.tail()

,ACC_MONTH,EST_COST_lag12,NBR_OTHER_lag12
162,2025-10-31,2656318.210,44
163,2025-11-30,3417666.784,32
164,2025-12-31,2071062.160,32
165,2026-01-31,3073348.775,36
166,2026-02-28,3173980.760,32
